In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install keras 
from tensorflow.keras import models,layers
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing import image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
batch = 32
classes = 'binary'

train='../input/dataset/train'
test='../input/dataset/test'
val='../input/dataset/val'


In [3]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
validation_datagen= image.ImageDataGenerator()

test_datagen= image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)


validation_generator = validation_datagen.flow_from_directory(
    val,
    target_size = (224,224),
    batch_size = batch,
    shuffle=True,
    class_mode = classes)

test_generator = test_datagen.flow_from_directory(
    test,
    target_size = (224,224),
    batch_size = batch,
    class_mode = classes)
class_names=validation_generator.class_indices
class_names
len(class_names)

Found 4946 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


2

In [4]:
base_for_model = resnet.ResNet101(weights='imagenet', input_shape=(224,224,3), include_top=False)
for layer in base_for_model.layers:
    layer.trainable = False


171446536/171446536 [==============================] - 1s 0us/step


In [5]:
model = Sequential()
model.add(base_for_model) 
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(GlobalMaxPooling2D()) 
model.add(BatchNormalization())
model.add(Dense(512, activation = 'relu')) 
model.add(BatchNormalization()) 
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 dense (Dense)               (None, 7, 7, 512)         1049088   
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 7, 7, 512)         262656    
                                                                 
 global_max_pooling2d (Globa  (None, 512)              0         
 lMaxPooling2D)                                                  
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2

In [6]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy','Precision','Recall','AUC']
)

In [8]:
history=model.fit(
    train_generator,
    epochs=20,
    batch_size=64,
    validation_data=validation_generator
)

Epoch 1/20
155/155 [==============================] - 127s 816ms/step - loss: 0.1225 - accuracy: 0.9533 - precision: 0.9513 - recall: 0.9570 - auc: 0.9904 - val_loss: 0.4366 - val_accuracy: 0.8583 - val_precision: 0.9032 - val_recall: 0.9130 - val_auc: 0.8975
Epoch 2/20
155/155 [==============================] - 128s 826ms/step - loss: 0.1168 - accuracy: 0.9527 - precision: 0.9567 - recall: 0.9498 - auc: 0.9916 - val_loss: 0.4829 - val_accuracy: 0.8167 - val_precision: 0.8365 - val_recall: 0.9457 - val_auc: 0.8740
Epoch 3/20
155/155 [==============================] - 128s 825ms/step - loss: 0.1162 - accuracy: 0.9551 - precision: 0.9606 - recall: 0.9506 - auc: 0.9908 - val_loss: 0.6581 - val_accuracy: 0.8417 - val_precision: 0.8411 - val_recall: 0.9783 - val_auc: 0.8418
Epoch 4/20
155/155 [==============================] - 126s 812ms/step - loss: 0.1069 - accuracy: 0.9602 - precision: 0.9599 - recall: 0.9618 - auc: 0.9927 - val_loss: 0.5416 - val_accuracy: 0.8583 - val_precision: 0.8641

In [ ]:
score=model.evaluate(validation_generator)
score